In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import basedosdados as bd
import re
import json
import string
import plotly
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import scipy.stats as stats
import sympy as sym
import itertools
import os
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
municipios = pd.read_csv("csv/municipios.csv")

orientacao_partidos = pd.read_csv("csv/orientacao_partidos.csv", error_bad_lines=False, delimiter=',')
orientacao_partidos =  orientacao_partidos.replace({ "—":"esquerda"})
orientacao_partidos['Nome'] = orientacao_partidos['Nome'].map(lambda x: x.upper())

with open('csv/bairros-geojson.json', encoding="utf8") as geojson:
    bairros = json.load(geojson)

In [ ]:
def dataAlreadyBeenGot(csvName):
    if os.stat(csvName).st_size == 0:
        return False
    else:
        return True

In [ ]:
if not dataAlreadyBeenGot('csv/secao_eleitoral.csv'):
    querySecaoEleitoral = """
        SELECT * FROM `basedosdados.br_tse_eleicoes.local_secao` AS LS
        WHERE LS.id_municipio={} AND LS.sigla_uf='RJ'
        """.format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_ibge'].values[0])

    secao_eleitoral = bd.read_sql(querySecaoEleitoral, billing_project_id='capacitacaoufrjanalytica')
    secao_eleitoral.to_csv("csv/secao_eleitoral.csv", index=False)
else:
    secao_eleitoral = pd.read_csv("csv/secao_eleitoral.csv")

In [ ]:
secao_eleitoral = secao_eleitoral[~(secao_eleitoral['melhor_urbano']).isnull()]

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    queryResultadosPartidoMunicipio = """
        SELECT MZ.ano, MZ.zona, MZ.secao, MZ.sigla_partido, MZ.cargo, MZ.turno, MZ.votos_nominais, MZ.votos_nao_nominais FROM `basedosdados.br_tse_eleicoes.resultados_partido_secao` AS MZ
        WHERE MZ.id_municipio_tse={} AND MZ.sigla_uf='RJ'
        ORDER BY MZ.ano ASC""".format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_tse'].values[0])

    dfResultadosPartidoMunicipio = bd.read_sql(queryResultadosPartidoMunicipio, billing_project_id='capacitacaoufrjanalytica')

In [ ]:
def neighhboorhoodCoordinateIsInside(zona, secao):
    zona_secao_df = secao_eleitoral[secao_eleitoral['zona'].eq(zona) &  secao_eleitoral['secao'].eq(secao)]
    
    if len(zona_secao_df) == 0:
        return 'nao informado'
    
    coords = zona_secao_df.values[0][5]
    array_coords = [float(x) for x in re.findall(r'[-+]?\d+\.\d+', zona_secao_df.iloc[0]['melhor_urbano'])]
    
    longitude = array_coords[0]
    latitude = array_coords[1]
    for e in bairros['features']:
        if (e['geometry']['type'] != 'MultiPolygon'):
            vertices = e['geometry']['coordinates'][0]
        else:
            vertices = e['geometry']['coordinates'][0][0]
        wn = 0;
        i = 0
        j = len(vertices) - 1
        while (i < len(vertices)):
            v = vertices[j];
            v1 = vertices[i];
            if ( v[0] <= longitude ) :
                if ( longitude < v1[0] ):  
                    if ( (v1[1] - v[1]) * (longitude - v[0]) - (v1[0] - v[0]) * (latitude - v[1]) > 0 ):
                        wn += 1;
            else:
                if( longitude >= v1[0] ):
                    if ( (v1[1] - v[1]) * (longitude - v[0]) - (v1[0] - v[0]) * (latitude - v[1]) < 0 ):
                        wn -= 1;
            j = i
            i += 1
    
        if (wn != 0):
            return e['properties']['NOME']

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    returnBairro = np.vectorize(neighhboorhoodCoordinateIsInside)
    bairrosFromZonaSecao = pd.DataFrame(list(itertools.product(dfResultadosPartidoMunicipio['zona'].unique(),  dfResultadosPartidoMunicipio['secao'].unique())),
                                        columns=['zona', 'secao'])
    bairrosFromZonaSecao['bairro'] = returnBairro(bairrosFromZonaSecao['zona'].values, bairrosFromZonaSecao['secao'].values)

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    dfResultadosPartidoMunicipio = pd.merge(dfResultadosPartidoMunicipio, bairrosFromZonaSecao, how='left', left_on=['secao', 'zona'], right_on = ['secao', 'zona'])
    dfResultadosPartidoMunicipio.replace(np.nan, 'nao informado', inplace=True)

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    dfResultadosPartidoMunicipio['sigla_partido'].replace({'PEN':'PATRIOTA', 'PATRI':'PATRIOTA', 
                                                           'PT DO B':'AVANTE', 'PT do B':'AVANTE', 
                                                           'PC do B': 'PCDOB', 'PC DO B': 'PCDOB',
                                                           'PRONA':'PL', 'PGT':'PL', 'PPB':'PP',
                                                           'PPS': 'CIDADANIA', 'PRN':'PTC',
                                                           'PRP': 'PATRIOTA', 'PSDC':'DC',
                                                           'PSN':'PODE', 'PST': 'PL', 'PTN':'PODE',
                                                           'PHS':'PODE', 'PRB':'REPUBLICANOS',
                                                           'PR': 'PL', 'PPL': 'PCDOB', 'SD': 'SOLIDARIEDADE',
                                                           'PAN':'PTB','PFL':'DEM', 'PMDB':'MDB'}, inplace=True)


In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    dfResultadosPartidoMunicipio = dfResultadosPartidoMunicipio.set_index(['ano', 'sigla_partido', 'bairro', 'cargo', 'turno']).groupby(level=(0, 1, 2, 3, 4)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
    dfResultadosPartidoMunicipio.to_csv("csv/resultado_eleicoes.csv", index=False)
else:
    dfResultadosPartidoMunicipio = pd.read_csv("csv/resultado_eleicoes.csv")

In [ ]:
votosTotaisPorPartido = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'sigla_partido']).groupby(level=(0, 1)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()

votosTotaisPorPartido['votos totais'] = votosTotaisPorPartido['votos_nominais'] + votosTotaisPorPartido['votos_nao_nominais']

In [ ]:
def returnPorcentagemDeVotos(partido, ano):
    votosPartido = votosTotaisPorPartido[votosTotaisPorPartido['sigla_partido'].eq(partido) & votosTotaisPorPartido['ano'].eq(ano)]['votos totais']
    votosTotaisBairro = votosTotaisPorPartido[votosTotaisPorPartido['ano'].eq(ano)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemVetorizada = np.vectorize(returnPorcentagemDeVotos)
votosTotaisPorPartido['desempenho'] = porcentagemVetorizada(votosTotaisPorPartido['sigla_partido'], votosTotaisPorPartido['ano']) 

In [ ]:
partido = ['PT', 'PSDB', 'PSL']
fig = px.line(votosTotaisPorPartido[votosTotaisPorPartido['sigla_partido'].isin(partido)], x="ano", y="desempenho", 
              color='sigla_partido',
              title='Desempenho dos partidos '+str(partido)[1:-1]+' X ano')
fig.show()

In [ ]:
votosTotaisPartidoPorBairro = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'bairro','sigla_partido']).groupby(level=(0, 1, 2)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()

votosTotaisPartidoPorBairro['votos totais'] = votosTotaisPartidoPorBairro['votos_nominais'] + votosTotaisPartidoPorBairro['votos_nao_nominais']

def returnPorcentagemPorBairro(bairro, partido, ano):
    votosPartido = votosTotaisPartidoPorBairro[votosTotaisPartidoPorBairro['bairro'].eq(bairro) & votosTotaisPartidoPorBairro['sigla_partido'].eq(partido) & votosTotaisPartidoPorBairro['ano'].eq(ano)]['votos totais']
    votosTotaisBairro = votosTotaisPartidoPorBairro[votosTotaisPartidoPorBairro['bairro'].eq(bairro) & votosTotaisPartidoPorBairro['ano'].eq(ano)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemPorBairroVetorizada = np.vectorize(returnPorcentagemPorBairro)

votosTotaisPartidoPorBairro['desempenho por bairro'] = porcentagemPorBairroVetorizada(votosTotaisPartidoPorBairro['bairro'], votosTotaisPartidoPorBairro['sigla_partido'], votosTotaisPartidoPorBairro['ano']) 

In [ ]:
partido = ['PT', 'PSL', 'PSOL', 'DEM']
bairro = 'Barra da Tijuca'
fig = px.line(votosTotaisPartidoPorBairro[votosTotaisPartidoPorBairro['sigla_partido'].isin(partido) & votosTotaisPartidoPorBairro['bairro'].eq(bairro)], 
              x="ano", y="desempenho por bairro", 
              color='sigla_partido',
              title='Desempenho dos partidos '+str(partido)[1:-1]+' no bairro '+bairro+' X ano')
fig.show()

In [ ]:
partido = 'PSOL'
bairro = ['Bangu', 'Laranjeiras', 'Freguesia (Jacarepaguá)', 'Cidade Universitária']
fig = px.line(votosTotaisPartidoPorBairro[votosTotaisPartidoPorBairro['sigla_partido'].eq(partido) & votosTotaisPartidoPorBairro['bairro'].isin(bairro)], 
              x="ano", y="desempenho por bairro", 
              color='bairro',
              title='Votos do partido '+partido+' nos bairros '+str(bairro)[1:-1]+' X ano')
fig.show()

In [ ]:
votosTotaisPorCargoPorPartido = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano','sigla_partido', 'cargo']).groupby(level=(0, 1, 2)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
votosTotaisPorCargoPorPartido['votos totais'] = votosTotaisPorCargoPorPartido['votos_nominais'] + votosTotaisPorCargoPorPartido['votos_nao_nominais']


In [ ]:
def returnPorcentagemDeVotosParaCargo(partido, ano, cargo):
    votosPartido = votosTotaisPorCargoPorPartido[votosTotaisPorCargoPorPartido['sigla_partido'].eq(partido) & votosTotaisPorCargoPorPartido['ano'].eq(ano) & votosTotaisPorCargoPorPartido['cargo'].eq(cargo)]['votos totais']
    votosTotaisBairro = votosTotaisPorCargoPorPartido[votosTotaisPorCargoPorPartido['ano'].eq(ano) & votosTotaisPorCargoPorPartido['cargo'].eq(cargo)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemPorCargoVetorizada = np.vectorize(returnPorcentagemDeVotosParaCargo)
votosTotaisPorCargoPorPartido['desempenho'] = porcentagemPorCargoVetorizada(votosTotaisPorCargoPorPartido['sigla_partido'], votosTotaisPorCargoPorPartido['ano'], votosTotaisPorCargoPorPartido['cargo']) 

In [ ]:
cargo = 'prefeito'
partido = ['DEM', 'PSOL', 'REPUBLICANOS']
fig = px.line(votosTotaisPorCargoPorPartido[votosTotaisPorCargoPorPartido['sigla_partido'].isin(partido) & votosTotaisPorCargoPorPartido['cargo'].eq(cargo)], 
              x="ano", y="desempenho", 
              color='sigla_partido',
              title='Desempenho dos partidos '+str(partido)[1:-1]+' para o cargo de '+cargo+' X ano')
fig.show()

In [ ]:
votosTotaisPorCargoPorPartidoPorBairro = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'bairro','sigla_partido', 'cargo']).groupby(level=(0, 1, 2, 3)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
votosTotaisPorCargoPorPartidoPorBairro['votos totais'] = votosTotaisPorCargoPorPartidoPorBairro['votos_nominais'] + votosTotaisPorCargoPorPartidoPorBairro['votos_nao_nominais']


In [ ]:
cargo = 'presidente'
partido = 'PSOL'
bairro = ['Taquara', 'Laranjeiras', 'Cachambi']
fig = px.line(votosTotaisPorCargoPorPartidoPorBairro[votosTotaisPorCargoPorPartidoPorBairro['sigla_partido'].eq(partido) & votosTotaisPorCargoPorPartidoPorBairro['cargo'].eq(cargo) & votosTotaisPorCargoPorPartidoPorBairro['bairro'].isin(bairro)], 
              x="ano", y="votos totais", 
              color='bairro',
              title='Votos do partido '+partido+' em '+str(bairro)[1:-1]+' para o cargo de '+cargo+' X ano')
fig.show()

In [ ]:
def returnOrientacaoPolitica(partido):
    return orientacao_partidos[orientacao_partidos['Nome'].eq(partido)]['Congresso em Foco (2019)[37]'].values[0]

In [ ]:
dfResultadosPartidoMunicipio['orientacao politica'] = dfResultadosPartidoMunicipio['sigla_partido'].map(lambda x: returnOrientacaoPolitica(x))

In [ ]:
votosPorOrientacaoPolitica = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'orientacao politica']).groupby(level=(0, 1)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
votosPorOrientacaoPolitica['votos totais'] = votosPorOrientacaoPolitica['votos_nominais'] + votosPorOrientacaoPolitica['votos_nao_nominais']

In [ ]:
def returnPorcentagemDeVotosPorOrientacao(orientacao, ano):
    votosPartido = votosPorOrientacaoPolitica[votosPorOrientacaoPolitica['orientacao politica'].eq(orientacao) & votosPorOrientacaoPolitica['ano'].eq(ano)]['votos totais']
    votosTotaisBairro = votosPorOrientacaoPolitica[votosPorOrientacaoPolitica['ano'].eq(ano)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemPorOrientacaoVetorizada = np.vectorize(returnPorcentagemDeVotosPorOrientacao)
votosPorOrientacaoPolitica['desempenho'] = porcentagemPorOrientacaoVetorizada(votosPorOrientacaoPolitica['orientacao politica'], votosPorOrientacaoPolitica['ano']) 

In [ ]:
fig = px.line(votosPorOrientacaoPolitica, x="ano", y="desempenho", color='orientacao politica',
             title='Desempenho de determinada ideologica política X ano')
fig.show()

In [ ]:
votosTotaisPartidoPorBairroPorOrientacao = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'bairro','orientacao politica']).groupby(level=(0, 1, 2)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()

votosTotaisPartidoPorBairroPorOrientacao['votos totais'] = votosTotaisPartidoPorBairroPorOrientacao['votos_nominais'] + votosTotaisPartidoPorBairroPorOrientacao['votos_nao_nominais']

def returnPorcentagemPorBairroPorOrientacao(bairro, orientacao, ano):
    votosPartido = votosTotaisPartidoPorBairroPorOrientacao[votosTotaisPartidoPorBairroPorOrientacao['bairro'].eq(bairro) & votosTotaisPartidoPorBairroPorOrientacao['orientacao politica'].eq(orientacao) & votosTotaisPartidoPorBairroPorOrientacao['ano'].eq(ano)]['votos totais']
    votosTotaisBairro = votosTotaisPartidoPorBairroPorOrientacao[votosTotaisPartidoPorBairroPorOrientacao['bairro'].eq(bairro) & votosTotaisPartidoPorBairroPorOrientacao['ano'].eq(ano)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemPorBairroPorOriencaoVetorizada = np.vectorize(returnPorcentagemPorBairroPorOrientacao)

votosTotaisPartidoPorBairroPorOrientacao['desempenho por bairro'] = porcentagemPorBairroPorOriencaoVetorizada(votosTotaisPartidoPorBairroPorOrientacao['bairro'], votosTotaisPartidoPorBairroPorOrientacao['orientacao politica'], votosTotaisPartidoPorBairroPorOrientacao['ano']) 

In [ ]:
bairro = 'Méier'
fig = px.line(votosTotaisPartidoPorBairroPorOrientacao[votosTotaisPartidoPorBairroPorOrientacao['bairro'].eq(bairro)], 
              x="ano", y="desempenho por bairro", 
              color='orientacao politica',
              title='Desempenho de determinada orientação política no bairro '+bairro+' X ano')
fig.show()

In [ ]:
orientacao = 'esquerda'
bairro = ['Bangu', 'Laranjeiras', 'Freguesia (Jacarepaguá)']
fig = px.line(votosTotaisPartidoPorBairroPorOrientacao[votosTotaisPartidoPorBairroPorOrientacao['orientacao politica'].eq(orientacao) & votosTotaisPartidoPorBairroPorOrientacao['bairro'].isin(bairro)], 
              x="ano", y="desempenho por bairro", 
              color='bairro',
              title='Desempenho do(a) '+orientacao+' nos bairros '+str(bairro)[1:-1]+' X ano')
fig.show()

In [ ]:
auxDf = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)]

def calcularResultadoAgrupadoPorAno(ano):
    bairrosArray = []
    n_votosArray = []
    partidosArray = []

    for bairro, frame in auxDf[auxDf['ano'].eq(ano)].groupby('bairro'):
        n_votos = np.max(frame['votos_nominais'])
        partido = auxDf[auxDf['votos_nominais'].eq(n_votos) & auxDf['bairro'].eq(bairro) & auxDf['ano'].eq(ano)]['sigla_partido'].values[0]
        
        n_votosArray.append(n_votos)
        partidosArray.append(partido)
        bairrosArray.append(bairro)

    dfAgrupadoResultadosPartidoMunicipio = pd.DataFrame([bairrosArray, n_votosArray, partidosArray], 
                              index=['bairro', 'votos', 'partido']).T
    
    dfAgrupadoResultadosPartidoMunicipio['ano'] = [ano]*len(dfAgrupadoResultadosPartidoMunicipio)
    
    return dfAgrupadoResultadosPartidoMunicipio

dfAgrupadoResultadosPartidoMunicipioPorAno = pd.DataFrame()
for a in dfResultadosPartidoMunicipio['ano'].unique():
    dfAgrupadoResultadosPartidoMunicipioPorAno = dfAgrupadoResultadosPartidoMunicipioPorAno.append(calcularResultadoAgrupadoPorAno(a))
dfAgrupadoResultadosPartidoMunicipioPorAno.reset_index(drop=True, inplace=True)

In [ ]:
ano = 2010
fig = px.choropleth_mapbox(dfAgrupadoResultadosPartidoMunicipioPorAno[dfAgrupadoResultadosPartidoMunicipioPorAno['ano'].eq(ano)], geojson=bairros, locations='bairro', color='partido',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['votos'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           title='Partido mais votado em cada bairro do Rio de Janeiro no ano '+str(ano),
                           zoom=9
                          )

fig.show()

In [ ]:
dfAgrupadoOrientacaoPoliticaMunicipioPorAno = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'orientacao politica', 'bairro']).groupby(level=(0, 1, 2)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
    
dfAgrupadoOrientacaoPoliticaMunicipioPorAno['votos totais'] = dfAgrupadoOrientacaoPoliticaMunicipioPorAno['votos_nominais'] + dfAgrupadoOrientacaoPoliticaMunicipioPorAno['votos_nao_nominais']

dfOrientacaoPoliticaMunicipioMaxVotos = pd.DataFrame()
for group, frame in dfAgrupadoOrientacaoPoliticaMunicipioPorAno.set_index(['ano', 'bairro']).groupby(level = (0, 1)):
    n_votos = np.max(frame['votos totais'])
    dfOrientacaoPoliticaMunicipioMaxVotos = dfOrientacaoPoliticaMunicipioMaxVotos.append(frame[frame['votos totais'].eq(n_votos)])

dfOrientacaoPoliticaMunicipioMaxVotos.reset_index(inplace=True)

In [ ]:
ano = 2018
fig = px.choropleth_mapbox(dfOrientacaoPoliticaMunicipioMaxVotos[dfOrientacaoPoliticaMunicipioMaxVotos['ano'].eq(ano)], 
                           geojson=bairros, locations='bairro', color='orientacao politica',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['votos totais'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           title='Orientação política mais votada em cada bairro do Rio de Janeiro no ano '+str(ano),
                           zoom=9
                          )

fig.show()

In [ ]:
if not dataAlreadyBeenGot('csv/filiados.csv'):
    dfFiliacaoPartidaria = pd.DataFrame()
    for ano in dfAgrupadoResultadosPartidoMunicipioPorAno['ano'].unique():
        queryFiliacaoPartidaria = """
            SELECT COUNT(*) AS n_filiados, FP.secao, FP.zona, FP.sigla_partido FROM `basedosdados.br_tse_filiacao_partidaria.filiacao_partidaria` AS FP
            WHERE FP.id_municipio_tse={} AND (FP.data_desfiliacao > '{}-12-30' OR (FP.situacao_registro = 'regular' AND FP.data_filiacao < '{}-01-01'))
            GROUP BY FP.zona, FP.sigla_partido, FP.secao
            ORDER BY FP.zona""".format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_tse'].values[0], ano, ano)

        dfResultadoQuery = bd.read_sql(queryFiliacaoPartidaria, billing_project_id='capacitacaoufrjanalytica')
        dfResultadoQuery['ano'] = [ano]*len(dfResultadoQuery)
        dfFiliacaoPartidaria = dfFiliacaoPartidaria.append(dfResultadoQuery)

In [ ]:
if not dataAlreadyBeenGot('csv/filiados.csv'):
    dfFiliacaoPartidaria = pd.merge(dfFiliacaoPartidaria, bairrosFromZonaSecao, how='left', left_on=['secao', 'zona'], right_on = ['secao', 'zona'])
    dfFiliacaoPartidaria.replace(np.nan, 'nao informado', inplace=True)


In [ ]:
if not dataAlreadyBeenGot('csv/filiados.csv'):
    dfFiliacaoPartidaria = dfFiliacaoPartidaria.set_index(['ano', 'sigla_partido', 'bairro']).groupby(level=(0, 1, 2)).agg({"n_filiados":np.sum}).reset_index()
    dfFiliacaoPartidaria.to_csv('csv/filiados.csv')
else:
    dfFiliacaoPartidaria = pd.read_csv("csv/filiados.csv")

In [ ]:
filiadosPorPartido = dfFiliacaoPartidaria.set_index(['ano', 'sigla_partido']).groupby(level=(0, 1)).agg({"n_filiados":np.sum}).reset_index()

In [ ]:
partido = ['MDB', 'PT', 'PSDB']
fig = px.line(filiadosPorPartido[filiadosPorPartido['sigla_partido'].isin(partido)], x="ano", y="n_filiados", 
              color = "sigla_partido",
              title='Filiados do '+str(partido)[1:-1]+' X ano')
fig.show()

In [ ]:
filiadosPartidoPorBairro = dfFiliacaoPartidaria.set_index(['ano', 'bairro','sigla_partido']).groupby(level=(0, 1, 2)).agg({"n_filiados":np.sum}).reset_index()

In [ ]:
partido = ['PT', 'PSL', 'PSOL', 'DEM']
bairro = 'Barra da Tijuca'
fig = px.line(filiadosPartidoPorBairro[filiadosPartidoPorBairro['sigla_partido'].isin(partido) & filiadosPartidoPorBairro['bairro'].eq(bairro)], 
              x="ano", y="n_filiados", 
              color='sigla_partido',
              title='Filiados dos partido '+str(partido)[1:-1]+' no(a) '+bairro+' X ano')
fig.show()

In [ ]:
dfFiliacaoPartidaria['orientacao politica'] = dfFiliacaoPartidaria['sigla_partido'].map(lambda x: returnOrientacaoPolitica(x))

In [ ]:
filiadosPorOrientacaoPolitica = dfFiliacaoPartidaria.set_index(['ano', 'orientacao politica']).groupby(level=(0, 1)).agg({"n_filiados":np.sum}).reset_index()

In [ ]:
fig = px.line(filiadosPorOrientacaoPolitica, x="ano", y="n_filiados", color='orientacao politica',
             title='Filiados de determinada ideologica política X ano')
fig.show()

In [ ]:
filiadosOrientacaoPorBairro = dfFiliacaoPartidaria.set_index(['ano', 'bairro','orientacao politica']).groupby(level=(0, 1, 2)).agg({"n_filiados":np.sum}).reset_index()

In [ ]:
orientacao = 'direita'
bairro = ['Barra da Tijuca', 'Taquara', 'Tanque', 'Copacabana']
fig = px.line(filiadosOrientacaoPorBairro[filiadosOrientacaoPorBairro['bairro'].isin(bairro) & filiadosOrientacaoPorBairro['orientacao politica'].eq(orientacao)], 
              x="ano", y="n_filiados", color='bairro',
             title='Filiados de partidos de '+orientacao+' nos(as) '+str(bairro)[1:-1]+' X ano')
fig.show()

In [ ]:
bairro = 'Barra da Tijuca'
fig = px.line(filiadosOrientacaoPorBairro[filiadosOrientacaoPorBairro['bairro'].eq(bairro)], 
              x="ano", y="n_filiados", color='orientacao politica',
             title='Filiados de determinada ideologica política no(a) '+bairro+' X ano')
fig.show()

In [ ]:
def calcularResultadoAgrupadoPorAno(ano):
    bairrosArray = []
    n_filiadosArray = []
    partidosArray = []
    
    for bairro, frame in dfFiliacaoPartidaria[dfFiliacaoPartidaria['ano'].eq(ano)].groupby('bairro'):
        n_filiados = np.max(frame['n_filiados'])
        partido = dfFiliacaoPartidaria[dfFiliacaoPartidaria['n_filiados'].eq(n_filiados) & dfFiliacaoPartidaria['bairro'].eq(bairro)]['sigla_partido'].values[0]
        
            
        n_filiadosArray.append(n_filiados)
        partidosArray.append(partido)
        bairrosArray.append(bairro)
        
    
    dfAgrupadoFiliacaoPartidaria = pd.DataFrame([bairrosArray, n_filiadosArray, partidosArray], 
                              index=['bairro', 'filiados', 'partido']).T
    
    dfAgrupadoFiliacaoPartidaria['ano'] = [ano]*len(dfAgrupadoFiliacaoPartidaria)
    
    return dfAgrupadoFiliacaoPartidaria

dfAgrupadoFiliacaoPartidariaPorAno = pd.DataFrame()
for a in dfFiliacaoPartidaria['ano'].unique():
    dfAgrupadoFiliacaoPartidariaPorAno = dfAgrupadoFiliacaoPartidariaPorAno.append(calcularResultadoAgrupadoPorAno(a))
dfAgrupadoFiliacaoPartidariaPorAno.reset_index(drop=True, inplace=True)                     

In [ ]:
ano = 2020
fig = px.choropleth_mapbox(dfAgrupadoFiliacaoPartidariaPorAno[dfAgrupadoFiliacaoPartidariaPorAno['ano'].eq(ano)], geojson=bairros, locations='bairro', color='partido',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['filiados'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
dfAgrupadoFiliacaoOrientacaoPoliticaPorAno = dfFiliacaoPartidaria.set_index(['ano', 'orientacao politica', 'bairro']).groupby(level=(0, 1, 2)).agg({"n_filiados":np.sum}).reset_index()

dfOrientacaoPoliticaMunicipioMaxFiliados = pd.DataFrame()
for group, frame in dfAgrupadoFiliacaoOrientacaoPoliticaPorAno.set_index(['ano', 'bairro']).groupby(level = (0, 1)):
    n_filiados = np.max(frame['n_filiados'])
    dfOrientacaoPoliticaMunicipioMaxFiliados = dfOrientacaoPoliticaMunicipioMaxFiliados.append(frame[frame['n_filiados'].eq(n_filiados)])

dfOrientacaoPoliticaMunicipioMaxFiliados.reset_index(inplace=True)

In [ ]:
ano = 2020
fig = px.choropleth_mapbox(dfOrientacaoPoliticaMunicipioMaxFiliados[dfOrientacaoPoliticaMunicipioMaxFiliados['ano'].eq(ano)], 
                           geojson=bairros, locations='bairro', color='orientacao politica',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['n_filiados'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
votosArray = []
anosArray = []
partidosArray = []
cargosArray = []
bairrosArray = []
for group, frame in dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].ne(2)].set_index(['ano', 'sigla_partido', 'cargo', 'bairro']).groupby(level=(0, 1, 2, 3)):
    n_votos = np.sum(frame['votos_nominais']) + np.sum(frame['votos_nao_nominais']) 
    votosArray.append(n_votos)
    anosArray.append(group[0])
    partidosArray.append(group[1])
    cargosArray.append(group[2])
    bairrosArray.append(group[3])

dfVotosAgrupados = pd.DataFrame([votosArray, anosArray, partidosArray, cargosArray, bairrosArray],
                               index=['votos', 'ano', 'partido', 'cargo', 'bairro']).T

In [ ]:
filiadosArray = []
anosArray = []
partidosArray = []
bairrosArray = []
for group, frame in dfFiliacaoPartidaria.set_index(['ano', 'sigla_partido', 'bairro']).groupby(level=(0, 1, 2)):
    n_filiados = np.sum(frame['n_filiados'])
    filiadosArray.append(n_filiados)
    anosArray.append(group[0])
    partidosArray.append(group[1])
    bairrosArray.append(group[2])
    
dfFiliadosAgrupados = pd.DataFrame([filiadosArray, anosArray, partidosArray, bairrosArray],
                               index=['filiados', 'ano', 'partido', 'bairro']).T

In [ ]:
new_df = pd.merge(dfFiliadosAgrupados, dfVotosAgrupados,  how='inner', left_on=['ano', 'partido', 'bairro'], right_on = ['ano', 'partido', 'bairro'])

In [ ]:
new_df['filiados'] = new_df['filiados'].values.astype(int)
new_df['votos'] = new_df['votos'].values.astype(int)

In [ ]:
dfWithoutCargos = new_df.set_index(['ano', 'partido', 'bairro']).groupby(level=(0, 1, 2)).agg({"votos":np.sum, "filiados":np.max}).reset_index()

In [ ]:
fig = px.scatter(dfWithoutCargos, x="filiados", y="votos", color='partido', hover_data=['ano', 'bairro'])
fig.show()

In [ ]:
dfPartidos = pd.DataFrame({'partido': dfWithoutCargos[:-1]['partido'].unique()})
dfPartidos['corr'] = dfPartidos['partido'].map(lambda x: stats.pearsonr(dfWithoutCargos[dfWithoutCargos['partido'].eq(x)]['filiados'], dfWithoutCargos[dfWithoutCargos['partido'].eq(x)]['votos'])[0])
dfPartidos.head()

In [ ]:
dfBairros = pd.DataFrame({'bairro': dfWithoutCargos['bairro'].unique()})
dfBairros['corr'] = dfBairros['bairro'].map(lambda x: stats.pearsonr(dfWithoutCargos[dfWithoutCargos['bairro'].eq(x)]['filiados'], dfWithoutCargos[dfWithoutCargos['bairro'].eq(x)]['votos'])[0])

dfBairros.head()

In [ ]:
dfCargos = pd.DataFrame({ 'cargo': new_df['cargo'].unique()})
dfCargos['corr'] = dfCargos['cargo'].map(lambda x: stats.pearsonr(new_df[new_df['cargo'].eq(x)]['filiados'], new_df[new_df['cargo'].eq(x)]['votos'])[0])

dfCargos.head()

In [ ]:
representacao_congresso = pd.read_csv('csv/representacao_congresso.csv')

representacao_congresso.columns = ['partido', 'Assentos no Senado', 'Assentos na Câmara']
representacao_congresso['Assentos no Senado'] = representacao_congresso['Assentos no Senado'].map(lambda x: np.int(x[:-4]))
representacao_congresso['Assentos na Câmara'] = representacao_congresso['Assentos na Câmara'].map(lambda x: np.int(x[:-6]))

representacao_congresso['partido'] = representacao_congresso['partido'].map(lambda x: x.upper())

In [ ]:
dfPartidos = pd.merge(dfPartidos, representacao_congresso,  how='inner', left_on=['partido'], right_on = ['partido'])

In [ ]:
dfPartidos['orientacao politica'] = dfPartidos['partido'].map(lambda x: orientacao_partidos[orientacao_partidos['Nome'].eq(x)]['Congresso em Foco (2019)[37]'].iloc[0])

In [ ]:
def retornarValorDiscretizado(x, minValue, maxValue, bins):
    for i in range(1, bins + 1):
        if x <= minValue + i * ((maxValue - minValue)/bins):
            return str(minValue + (i-1) * ((maxValue - minValue)/bins))+ " - " + str(minValue + i * ((maxValue - minValue)/bins))

def discretizar(i):
    minValue = np.min(i)
    maxValue = np.max(i)
    bins = int(np.sqrt(len(i)))
            
    return i.map(lambda v:retornarValorDiscretizado(v, minValue, maxValue, bins))

dfPartidos['corr discretizada'] = discretizar(dfPartidos['corr'])

In [ ]:
def calcularIndiceDeGiniParaValor(df, resultado, valor, coluna):
    gini = 0
    for value in df[resultado].unique():
        pi = len(df[df[coluna].eq(valor) & df[resultado].eq(value)])/len(df[df[coluna].eq(valor)])
        gini += pi * pi
    gini = 1 - gini
    return gini
    
def calcularIndiceDeGini(df, resultado, coluna):
    values = df[coluna].unique()
    gini = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        gini += calcularIndiceDeGiniParaValor(df, resultado, i, coluna) * pi
    
    return gini

def calcularEntropia(df, coluna):
    values = df[coluna].unique()
    entropia = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        entropia += pi*np.log2(pi)
    return -entropia

def calcularGanho(df, resultado, coluna):
    values = df[coluna].unique()
    ganho = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        ganho += pi * calcularEntropia(df[df[coluna].eq(i)], resultado)
    ganho = calcularEntropia(df, resultado) - ganho
    return ganho

In [ ]:
partidosGini = pd.DataFrame()

for c in ['orientacao politica', 'Assentos no Senado', 'Assentos na Câmara']:
    partidosGini = partidosGini.append({'coluna': c, 
                                        'gini': calcularIndiceDeGini(dfPartidos, 'corr discretizada', c), 
                                        'informação mutua': calcularGanho(dfPartidos, 'corr discretizada', c)},
                                       ignore_index=True)

In [ ]:
fig = px.bar(partidosGini, x='coluna', y='gini', color='coluna',
            title='Indice de gini por coluna dos Partidos')
fig.show()

In [ ]:
fig = px.bar(partidosGini, x='coluna', y='informação mutua', color='coluna',
            title='Informação Mútua por coluna dos Partidos')
fig.show()

In [ ]:
partidosCorrMediaOrientacaoPolitica = dfPartidos.groupby('orientacao politica').agg({'corr': np.average}).reset_index()

fig = px.bar(partidosCorrMediaOrientacaoPolitica, x='orientacao politica', y='corr', color='orientacao politica',
            title='Correlação media por orientacao politica')
fig.show()

In [ ]:
partidosCorrMediaAssentosNoSenado = dfPartidos.groupby('Assentos no Senado').agg({'corr': np.average}).reset_index()

fig = px.bar(partidosCorrMediaAssentosNoSenado, x='Assentos no Senado', y='corr', color='Assentos no Senado',
            title='Correlação media por Assentos no Senado')
fig.show()

In [ ]:
partidosCorrMediaAssentosNoSenadoCongresso = dfPartidos.groupby('Assentos na Câmara').agg({'corr': np.average}).reset_index()

fig = px.bar(partidosCorrMediaAssentosNoSenadoCongresso, x='Assentos na Câmara', y='corr', color='Assentos na Câmara',
            title='Correlação media por Assentos na Câmara')
fig.show()

In [ ]:
bairro_idh = pd.read_csv('csv/bairro_idh.csv')
bairro_idh = bairro_idh.iloc[1:]
bairro_idh.columns = ['N', 'bairro', 'exp de vida', 'alfabetizacao', 'frequencia escolar', 'renda per capita' ,'IL', 'IE', 'IR', 'IDH']
bairro_idh = bairro_idh.replace(to_replace ='\[\w\]', value = '', regex = True)
bairro_idh.replace(to_replace =',', value = '.', regex = True, inplace=True)
        
cols=[i for i in bairro_idh.columns if i not in ["N","bairro"]]
for col in cols:
    bairro_idh[col]=pd.to_numeric(bairro_idh[col])

In [ ]:
dfBairros = pd.merge(dfBairros, bairro_idh,  how='inner', left_on=['bairro'], right_on = ['bairro'])

In [ ]:
bairro_ids = pd.read_csv('csv/bairro_ids.csv')
bairro_ids.columns = ['N', 'bairro', 'R A', 'regiao', 'IDS']
bairro_ids.replace(to_replace =',', value = '.', regex = True, inplace=True)
bairro_ids['IDS'] = pd.to_numeric(bairro_ids['IDS']) 

In [ ]:
dfBairros = pd.merge(dfBairros, bairro_ids.loc[:, ~bairro_ids.columns.isin(['N'])],  how='inner', left_on=['bairro'], right_on = ['bairro'])

In [ ]:
dfBairros['exp de vida discretizada'] = discretizar(dfBairros['exp de vida'])
dfBairros['alfabetizacao discretizada'] = discretizar(dfBairros['alfabetizacao'])
dfBairros['frequencia escolar discretizada'] = discretizar(dfBairros['frequencia escolar'])
dfBairros['renda per capita discretizada'] = discretizar(dfBairros['renda per capita'])
dfBairros['corr discretizada'] = discretizar(dfBairros['corr'])

In [ ]:
bairrosGini = pd.DataFrame()

for c in ['exp de vida discretizada', 'alfabetizacao discretizada', 'frequencia escolar discretizada', 'renda per capita discretizada', 'regiao']:
    bairrosGini = bairrosGini.append({'coluna': c, 
                                        'gini': calcularIndiceDeGini(dfBairros, 'corr discretizada', c)}, 
                                       ignore_index=True)

In [ ]:
fig = px.bar(bairrosGini, x='coluna', y='gini', color='coluna',
            title='Indice de gini por coluna dos Bairros')
fig.show()

In [ ]:
bairrosCorrMediaoRegiao = dfBairros.groupby('regiao').agg({'corr': np.average}).reset_index()

fig = px.bar(bairrosCorrMediaoRegiao, x='regiao', y='corr', color='regiao',
            title='Correlação media por região')
fig.show()

In [ ]:
bairrosCorrMediaExpDeVida = dfBairros.groupby('exp de vida discretizada').agg({'corr': np.average}).reset_index()

fig = px.bar(bairrosCorrMediaExpDeVida, x='exp de vida discretizada', y='corr', color='exp de vida discretizada',
            title='Correlação media por Expectativa de Vida')
fig.show()

In [ ]:
bairrosCorrMediaRenda = dfBairros.groupby('renda per capita discretizada').agg({'corr': np.average}).reset_index()


fig = px.bar(bairrosCorrMediaRenda, x='renda per capita discretizada', y='corr', color='renda per capita discretizada',
            title='Correlação media por Renda per Capita')
fig.show()

In [ ]:
bairrosCorrMediaAlfabetizacao = dfBairros.groupby('alfabetizacao discretizada').agg({'corr': np.average}).reset_index()


fig = px.bar(bairrosCorrMediaAlfabetizacao, x='alfabetizacao discretizada', y='corr', color='alfabetizacao discretizada',
            title='Correlação media por Alfabetização')
fig.show()

In [ ]:
bairrosCorrMediaFrequenciaEscolar = dfBairros.groupby('frequencia escolar discretizada').agg({'corr': np.average}).reset_index()


fig = px.bar(bairrosCorrMediaFrequenciaEscolar, x='frequencia escolar discretizada', y='corr', color='frequencia escolar discretizada',
            title='Correlação media por Frequência Escolar')
fig.show()

In [ ]:
reg = LinearRegression().fit(dfWithoutCargos['filiados'].values.reshape(-1, 1),dfWithoutCargos['votos'].values )

In [ ]:
reg.score(dfWithoutCargos['filiados'].values.reshape(-1, 1),dfWithoutCargos['votos'].values)

In [ ]:
fig = px.scatter(dfWithoutCargos, x="filiados", y="votos", color='partido', hover_data=['ano', 'bairro'])

fig.add_trace(
    go.Scatter(
        x=dfWithoutCargos['filiados'].values,
        y=reg.predict(dfWithoutCargos['filiados'].values.reshape(-1, 1)),
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=False)
)
fig.show()

In [ ]:
regDict = {}
for partido in dfWithoutCargos['partido'].unique():
    regDict[partido] = LinearRegression().fit(dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['filiados'].values.reshape(-1, 1),dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['votos'].values )

In [ ]:
dfScorePartido = pd.DataFrame({'partido': dfWithoutCargos['partido'].unique()})
dfScorePartido['score'] = [regDict[partido].score(dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['filiados'].values.reshape(-1, 1), dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['votos'].values) for partido in dfWithoutCargos['partido'].unique()]
dfScorePartido

In [ ]:
partido = 'NOVO'
fig = px.scatter(dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)], x="filiados", y="votos", color='bairro', hover_data=['ano', 'bairro'])

fig.add_trace(
    go.Scatter(
        x=dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['filiados'].values,
        y=regDict[partido].predict(dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['filiados'].values.reshape(-1, 1)),
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=False)
)
fig.show()

In [ ]:
for bairro in dfWithoutCargos['bairro'].unique():
    regDict[bairro] = LinearRegression().fit(dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['filiados'].values.reshape(-1, 1),dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['votos'].values )

In [ ]:
dfScoreBairro = pd.DataFrame({'bairro': dfWithoutCargos['bairro'].unique()})
dfScoreBairro['score'] = [regDict[bairro].score(dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['filiados'].values.reshape(-1, 1), dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['votos'].values) for bairro in dfWithoutCargos['bairro'].unique()]
dfScoreBairro.head()

In [ ]:
bairro = 'Anil'
fig = px.scatter(dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)], x="filiados", y="votos", color='partido', hover_data=['ano', 'bairro'])

fig.add_trace(
    go.Scatter(
        x=dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['filiados'].values,
        y=regDict[bairro].predict(dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['filiados'].values.reshape(-1, 1)),
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=False)
)
fig.show()

In [ ]:
def desempenhoVotos(orientacao, bairro):
    votosPorBairroPorOrientacao = dfForKmeans[dfForKmeans['orientacao politica'].eq(orientacao) & dfForKmeans['bairro'].eq(bairro)]['votos']
    votosPorBairro = dfForKmeans[dfForKmeans['bairro'].eq(bairro)]['votos'].sum()
    
    return votosPorBairroPorOrientacao/votosPorBairro

def desempenhoFiliados(orientacao, bairro):
    filiadosPorBairroPorOrientacao = dfForKmeans[dfForKmeans['orientacao politica'].eq(orientacao) & dfForKmeans['bairro'].eq(bairro)]['filiados']
    filiadosPorBairro = dfForKmeans[dfForKmeans['bairro'].eq(bairro)]['filiados'].sum()
    
    return filiadosPorBairroPorOrientacao/filiadosPorBairro

desempenhoVotos = np.vectorize(desempenhoVotos)
desempenhoFiliados = np.vectorize(desempenhoFiliados)

dfForKmeans = dfWithoutCargos.copy()
dfForKmeans['orientacao politica'] = dfForKmeans['partido'].map(lambda x: returnOrientacaoPolitica(x))
dfForKmeans = dfForKmeans.set_index(['bairro', 'orientacao politica']).groupby(level=(0, 1)).agg({"votos":np.sum, "filiados":np.max}).reset_index()
dfForKmeans['desempenho votos'] = desempenhoVotos(dfForKmeans['orientacao politica'].values, dfForKmeans['bairro'].values)
dfForKmeans['desempenho filiados'] = desempenhoFiliados(dfForKmeans['orientacao politica'].values, dfForKmeans['bairro'].values)

dfForKmeans = pd.pivot_table(dfForKmeans, values=['desempenho votos', 'desempenho filiados'], index=['bairro'], columns=['orientacao politica'], aggfunc={np.sum})
dfForKmeans.reset_index(inplace=True)
dfForKmeans.columns = ['bairro', 'filiados_centro', 'filiados_direita', 'filiados_esquerda', 'votos_centro', 'votos_direita' , 'votos_esquerda']

dfForKmeans[['votos_esquerda', 'votos_direita', 'votos_centro', 'filiados_esquerda', 'filiados_direita', 'filiados_centro']] = StandardScaler().fit_transform(dfForKmeans[['votos_esquerda', 'votos_direita', 'votos_centro', 'filiados_esquerda', 'filiados_direita', 'filiados_centro']])

In [ ]:
kmeans = KMeans(n_clusters=3).fit(dfForKmeans[dfForKmeans.columns.difference(['bairro'])].values)
dfForKmeans['labels'] = kmeans.labels_

In [ ]:
fig = px.scatter_3d(dfForKmeans,
                 x="votos_esquerda", y="votos_direita", z='votos_centro',color='labels', hover_data=['bairro'])
fig.show()

In [ ]:
fig = px.scatter_3d(dfForKmeans,
                 x="filiados_esquerda", y="filiados_direita", z='filiados_centro', color='labels', hover_data=['bairro'])
fig.show()

In [ ]:
fig = px.choropleth_mapbox(dfForKmeans, 
                           geojson=bairros, locations='bairro', color='labels',
                           featureidkey="properties.NOME",
                           mapbox_style="carto-positron",
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
kmeansVotos = KMeans(n_clusters=3).fit(dfForKmeans[dfForKmeans.columns.difference(['bairro', 'filiados_centro', 'filiados_direita', 'filiados_esquerda'])].values)
dfForKmeans['labels votos'] = kmeansVotos.labels_

In [ ]:
fig = px.scatter_3d(dfForKmeans,
                 x="votos_esquerda", y="votos_direita", z='votos_centro',color='labels votos', hover_data=['bairro'])
fig.show()

In [ ]:
fig = px.choropleth_mapbox(dfForKmeans, 
                           geojson=bairros, locations='bairro', color='labels votos',
                           featureidkey="properties.NOME",
                           mapbox_style="carto-positron",
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
kmeansFiliados = KMeans(n_clusters=3).fit(dfForKmeans[dfForKmeans.columns.difference(['bairro', 'votos_centro', 'votos_direita' , 'votos_esquerda'])].values)
dfForKmeans['labels filiados'] = kmeansFiliados.labels_

In [ ]:
fig = px.scatter_3d(dfForKmeans,
                 x="filiados_esquerda", y="filiados_direita", z='filiados_centro', color='labels filiados', hover_data=['bairro'])
fig.show()

In [ ]:
fig = px.choropleth_mapbox(dfForKmeans, 
                           geojson=bairros, locations='bairro', color='labels filiados',
                           featureidkey="properties.NOME",
                           mapbox_style="carto-positron",
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
np.mean(dfForKmeans[dfForKmeans['labels votos'].eq(1)]['votos_esquerda'])

In [ ]:
np.mean(dfForKmeans[dfForKmeans['labels votos'].eq(1)]['votos_direita'])

In [ ]:
np.mean(dfForKmeans[dfForKmeans['labels votos'].eq(1)]['votos_centro'])

In [ ]:
np.mean(dfForKmeans[dfForKmeans['labels votos'].eq(0)]['votos_esquerda'])

In [ ]:
np.mean(dfForKmeans[dfForKmeans['labels votos'].eq(0)]['votos_direita'])

In [ ]:
np.mean(dfForKmeans[dfForKmeans['labels votos'].eq(0)]['votos_centro'])

In [ ]:
np.mean(dfForKmeans[dfForKmeans['labels votos'].eq(2)]['votos_esquerda'])

In [ ]:
np.mean(dfForKmeans[dfForKmeans['labels votos'].eq(2)]['votos_direita'])

In [ ]:
np.mean(dfForKmeans[dfForKmeans['labels votos'].eq(2)]['votos_centro'])